In [113]:
import urllib
import bs4 
import numpy as np
import pandas as pd
import re
import csv


In [114]:
url = "https://www.goodreads.com/shelf/show/middle-grade"
source = urllib.request.urlopen(url).read()
soup = bs4.BeautifulSoup(source, 'html.parser')


In [115]:
#Extract the total number of books in middle-grade shelf
book_num_string = soup.find('span', class_='smallText').string.rstrip()
#print(book_num_string)
book_num_fields = book_num_string.split(' ')[-1].split(',')
book_num = int("".join(book_num_fields))
print("book number:" + str(book_num))

#Compute the number of pages we need to crawl to get all books
num_per_page = 50
total_pages = int(book_num / num_per_page) + 1
print("Number of web pages: " + str(total_pages))

domain_name = "https://www.goodreads.com"

book number:45121
Number of web pages: 903


In [116]:
def extract_book_link_one_page(page_num):
    """Extract the book links in the page with index page_num"""
    
    url = "https://www.goodreads.com/shelf/show/middle-grade?page=" + str(page_num)
    html_page = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(html_page, 'html.parser')
    
    #Extract links for books
    links_of_one_page = [domain_name + a.get("href") for a in soup.find_all('a', class_= "bookTitle")]
    return links_of_one_page

links = extract_book_link_one_page(1)
print(len(links))


50


In [8]:
print(links[0])

https://www.goodreads.com/book/show/28187.The_Lightning_Thief


Initialize a pandas DataFrame to store books information scraped from webpages

In [93]:
#column_names = ['Title', 'Author', 'Ave_rating', 'Rating_cnt', 'Review_cnt', 'Price', 'Year', 'Pages', 'Publisher']
column_names = ['Title', 'Author', 'Ave_rating', 'Rating_cnt', 'Review_cnt', 'Year', 'Pages', 'Publisher']
books_df = pd.DataFrame(columns=column_names)

In [89]:
def get_price_from_amazon(amazon_link):
    """Get the price from the amazon page"""
    amazon_page = urllib.request.urlopen(amazon_link).read()
    soup = bs4.BeautifulSoup(amazon_page, 'html.parser')
    
    price = soup.find('span', class_="a-size-base a-color-price a-color-price")
    print(price)
    


In [117]:
def parse_one_book(book_link):
    """parse one book and store information in pandas object"""
    book_page = urllib.request.urlopen(book_link).read()
    soup = bs4.BeautifulSoup(book_page, 'html.parser')
    
    title = soup.find(id="bookTitle").string.strip()
    author = soup.find('a', class_= "authorName").find('span').string
    ave_rating = float(soup.find("span", itemprop="ratingValue").string)
    rating_cnt = int(soup.find("meta", itemprop = "ratingCount").get("content"))
    review_cnt = int(soup.find("meta", itemprop = "reviewCount").get("content"))
    
   # amazon_link = domain_name + soup.find(id="buyButton").get("href")
   # price = get_price_from_amazon(amazon_link)
    
    details = str(soup.find(id="details").find_all("div")[1])
    year = re.findall("\d{3,4}", details)[0]
    publisher_line = details.split("\n")[3]
    m = re.search(r"by\s+(.+)", publisher_line);
    publisher = m.group(1)
    
    pages = int(soup.find("span", itemprop = "numberOfPages").string.split(" ")[0])
    
    #book = [title, author, ave_rating, rating_cnt, review_cnt, price, year, pages, publisher]
    book = [title, author, ave_rating, rating_cnt, review_cnt, year, pages, publisher]
    return book
     

parse_one_book(links[0])

['The Lightning Thief',
 'Rick Riordan',
 4.24,
 1681440,
 52520,
 '2006',
 377,
 'Disney Hyperion Books']

Put the books information of one page in a pandas dataframe and append them to a csv file

In [119]:
column_names = ['Title', 'Author', 'Ave_rating', 'Rating_cnt', 'Review_cnt', 'Year', 'Pages', 'Publisher']


def scrape_books(total_pages, export_csv_file):
    """Scrape books from all pages"""
    with open(export_csv_file, 'a') as write_obj:
        writer = csv.writer(write_obj)
        writer.writerow(column_names)
    for page_num in np.arange(1, (total_pages + 1)):
        print("Processing page " + str(page_num) + "...")
        with open(export_csv_file, 'a') as write_obj:
            writer = csv.writer(write_obj)
    
            links_of_one_page = extract_book_link_one_page(page_num)
            for link in links_of_one_page:
                book_info = parse_one_book(link)
                print(book_info)
                writer.writerow(book_info)
            
            

export_csv_file = "/Users/hao/code/GoodReadsBooksAnalysis/middle_grade_books.page1.csv"
scrape_books = scrape_books(1, export_csv_file)

Processing page 1...
['The Lightning Thief', 'Rick Riordan', 4.24, 1681440, 52520, '2006', 377, 'Disney Hyperion Books']
["Harry Potter and the Sorcerer's Stone", 'J.K. Rowling', 4.46, 5848097, 93982, '1997', 320, 'Scholastic Inc']
['Harry Potter and the Chamber of Secrets', 'J.K. Rowling', 4.41, 2267965, 43326, '1999', 341, 'Arthur A. Levine Books / Scholastic Inc.']
['Wonder', 'R.J. Palacio', 4.45, 577657, 59596, '2012', 315, 'Alfred A. Knopf']
['The Sea of Monsters', 'Rick Riordan', 4.24, 633376, 23028, '2006', 279, 'Hyperion Books']
['Harry Potter and the Prisoner of Azkaban', 'J.K. Rowling', 4.55, 2316208, 45509, '2004', 435, 'Scholastic Inc.']
["The Titan's Curse", 'Rick Riordan', 4.35, 582789, 17749, '2007', 320, 'Puffin Books']
['The Battle of the Labyrinth', 'Rick Riordan', 4.39, 613725, 16262, '2008', 361, 'Hyperion Books for Children']
['The Last Olympian', 'Rick Riordan', 4.51, 569812, 20025, '2009', 381, 'Disney-Hyperion Books']
['A Wrinkle in Time', "Madeleine L'Engle", 4

Empty DataFrame
Columns: [Title, Author, Ave_rating, Rating_cnt, Review_cnt, Year, Pages, Publisher]
Index: []
